# Example 1: MNIST dataset in tensorflow

In [2]:
import onnx
import onnxruntime as ort
import numpy as np
from onnx import shape_inference
import os
import sys
import torch
import streamease as se

load data

In [ ]:
from helpers.data_loader import load_mnist_data

# Example usage:
input_shape = (31, 32)

# Call the functions with the instance
train_data, test_data,noisy_train_data, noisy_test_data = load_mnist_data(row=input_shape[0], column=input_shape[1])

Let's create a simple TCN model

In [4]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def Net(input_shape, padding='causal'):
    x_input = Input(shape=input_shape, name='input')

    # TCN layers
    x = Conv1D(filters=36, kernel_size=3, dilation_rate=1, padding=padding, activation='relu')(x_input)
    x = Conv1D(filters=40, kernel_size=5, dilation_rate=2, padding=padding, activation='relu')(x)
    out = Conv1D(filters=input_shape[1], kernel_size=2, dilation_rate=4, padding=padding)(x)

    # out = Dense(28, activation='sigmoid')(x)  # Output layer

    model = Model(inputs=[x_input], outputs=[out])
    model.compile(optimizer=Adam(learning_rate=0.002), loss='mean_squared_error')
    return model


Define Train and Test functions

In [5]:
def train(model, x_train , y_train, x_validate, y_validate, epochs, batch_size):
        model.fit(
                x=x_train,
                y=y_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(x_validate, y_validate),
        )
        return model
def test(model, x_validate, test_size):
        predictions = model.predict(x_validate[:test_size])
        return predictions

run and test the network 

In [ ]:
model = Net(input_shape, padding='causal')
model.summary()
epochs = 1
batch_size = 128
model = train(model, noisy_train_data,train_data, noisy_test_data, test_data,epochs, batch_size)

In [7]:
del noisy_train_data
del train_data
del test_data

In [ ]:
test_size = 1
predictions  = test(model, noisy_test_data, test_size)
print(f"prediction{predictions}")

from helpers.preprocessing import display
display(noisy_test_data[:test_size], predictions)

In [9]:
import tensorflow as tf
import tf2onnx
import os
from tensorflow import keras

def save_onnx(model, path='.', onnx_filename='original_model'):
    # Construct the full path to save the ONNX model
    full_path = os.path.join(path, f"{onnx_filename}.onnx")

    # Assuming `model` is your Keras model
    # Retrieve the input tensor from the first layer
    input_tensor = model.layers[0]._input_tensor
    input_signature = tf.TensorSpec(
        name=input_tensor.name, shape=input_tensor.shape, dtype=input_tensor.dtype
    )
    
    # Get the output name from the last layer
    output_name = model.layers[-1].name

    # Wrap the model in a tf.function with the input signature
    @tf.function(input_signature=[input_signature])
    def _wrapped_model(input_data):
        return {output_name: model(input_data)}

    # Convert the wrapped model to ONNX
    tf2onnx.convert.from_function(
        function=_wrapped_model,
        input_signature=[input_signature],
        output_path=full_path
    )

    print(f"ONNX model saved to {full_path}")

In [ ]:

save_onnx(model, path='.', onnx_filename='original_model')

# Convert it to streaming model

In [11]:

# Load the ONNX model
onnx_model = onnx.load("original_model.onnx")

In [ ]:
from streamease.onnx_streamer.streamer import StreamingConverter

streaming =  StreamingConverter(onnx_model, 1)


streaming.run()

streaming.print_info()
streaming.save_streaming_onnx('.', onnx_filename='streaming_model')

In [13]:

# Load the modified ONNX model
modified_model = onnx.load("streaming_model.onnx")

# Infer shapes
inferred_model = shape_inference.infer_shapes(modified_model)

# Save the inferred model (optional)
onnx.save(inferred_model, "inferred_model.onnx")

# Inference using streaming onnx

check the non-streaming onnx

In [ ]:

onnx_model = onnx.load("original_model.onnx")
onnx.checker.check_model(onnx_model)

# Load the ONNX model
model_path = "original_model.onnx"
ort_sess  = ort.InferenceSession(model_path)


input_data = noisy_test_data[:1].astype(np.float32)
# print(f"input data {noisy_test_data[:test_size]}")

# Run the model
output = ort_sess .run(None, {'input': input_data})

# 'output' contains the model's output (replace 'output_name' with the actual output name in your model)
print("Model output:", output[0].shape)
display(input_data, output[0])
print(output[0])

In [ ]:
from utils.onnx_inference.streaming_inference import Inference

original_model ="original_model.onnx"
streaming_model = "streaming_model.onnx" 
s_test = Inference(streaming_model,time_steps=1, receptive_field=14, causal=True)

s_test.init_buffers()

str_output = s_test.run(input_data)
display(input_data[:1],str_output[:1])
print(str_output[:1].shape)
print(str_output[:1])

If we want to check the non-causal network, we need to create a non-causal model and then transfer the weights 

In [ ]:
for layer in model.layers:
    print(layer.name)
    for weight in layer.weights:
        print(weight.name, weight.shape)

In [ ]:
nc_model = Net(input_shape, padding="valid")
nc_model.set_weights(model.get_weights())
nc_model.summary()

In [ ]:
save_onnx(nc_model, path='.', onnx_filename='nc_original_model')

In [ ]:
# Load the ONNX model
onnx_model = onnx.load("nc_original_model.onnx")

streaming =  StreamingConverter(onnx_model, 1)


streaming.run()

streaming.print_info()
streaming.save_streaming_onnx('.', onnx_filename='streaming_model')

In [ ]:

onnx_model = onnx.load("nc_original_model.onnx")
onnx.checker.check_model(onnx_model)

# Load the ONNX model
model_path = "nc_original_model.onnx"
ort_sess  = ort.InferenceSession(model_path)


input_data = noisy_test_data[:1].astype(np.float32)
# print(f"input data {noisy_test_data[:test_size]}")

# Run the model
output = ort_sess .run(None, {'input': input_data})

# 'output' contains the model's output (replace 'output_name' with the actual output name in your model)
print("Model output:", output[0].shape)
display(input_data, output[0])
print(output[0])

In [ ]:
original_model ="nc_original_model.onnx"
streaming_model = "streaming_model.onnx" 
s_test = Inference(streaming_model,time_steps=1, receptive_field=14, causal=False)

s_test.init_buffers()

str_output = s_test.run(input_data)
display(input_data[:1],str_output[:1])
print(str_output[:1].shape)
print(str_output[:1])

# NNTOOL

In [22]:

from nntool.api import NNGraph
from nntool.api.utils import model_settings, quantization_options, tensor_plot
# import logging
# nntool_log = logging.getLogger('nntool')
# nntool_log.setLevel(logging.ERROR)

In [ ]:
model = NNGraph.load_graph('original_model.onnx', use_onnx_names=True)

model.adjust_order()
model.fusions('scaled_match_group')
    
# Model show returns a table of information on the Graph
print(model.show())

# Model draw can open or save a PDF with a visual representation of the graph
# model.draw()

In [ ]:
data = noisy_test_data[:1].astype(np.float32)
print(data.shape)
output=model.execute(np.transpose(data, (0, 2, 1)))[-1] #[0]

for item in output:
    item = np.transpose(item, (0, 2, 1))
    print(item)
    display(data, item[:1])


In [ ]:
from utils.nntool_inference.streaming_inference import Inference

# Set printing options to display larger numbers without scientific notation
np.set_printoptions(precision=8, suppress=False, threshold=np.inf)

streaming_model_path = "streaming_model.onnx" 
s_model = NNGraph.load_graph(streaming_model_path, use_onnx_names=True)
s_model.adjust_order()
# s_model.draw()
# The equivalent of the fusions --scale8 command. The fusions method can be given a series of fusions to apply
# fusions('name1', 'name2', etc)
s_model.fusions('scaled_match_group')

s_test = Inference( s_model, streaming_model_path, receptive_field=14, time_steps=1, causal=True)

s_test.init_buffers()
input_data = noisy_test_data[:1].astype(np.float32)
# input_data = np.reshape(input_data, (1,32,28))

# input_data = np.transpose(input_data, (0,2,1))
print(input_data[:1].shape)
nn_str_output = s_test.run(input_data, i_transpose=True, o_transpose=False)
display(input_data[:1],nn_str_output[:1])
for i in range(len(nn_str_output)):
    print("#######################################")
    # print("golden model", output[i])
    print("streaming model", nn_str_output[i])